Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем словари, ключами которых являются координаты точек s, а значениями - минимальное хроматическое число (dict_det), максимальное запрещенное расстоянием (dict_dist), список матриц (dict_s), соответствующих данной точке s, хроматическому числу и запрещенному расстоянию
- составляем датафрейм (df), обьединяя все эти словари
- составляем список матриц для каждого подходящего хроматического числа (если запрещенное расстояние больше 1)

In [1]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from copy import deepcopy
from collections import defaultdict

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

In [3]:
points_count = 8

coords4 = []
digits = range(- (points_count // 4), points_count - (points_count // 4))

for var in itertools.product(digits, repeat=4):
    coords4.append((grid.T).dot(var).tolist())
    
#coords4 # координаты центров

In [4]:
# строим диаграмму Вороного

vor4 = Voronoi(coords4)

In [ ]:
# вершины диаграммы Вороного

vor4.vertices

In [ ]:
# регионы

vor4.regions

In [5]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        
print(count)

1297


In [6]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество =', count_r)

количество вершин = 24 количество = 1297


In [7]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 not in vor4.regions[i]: # регион должен быть конечным
        l = 0
        
        # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
        for j in range(len_r):
            l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

        if l < sum_dist_min:
            sum_dist_min = l
            v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

суммарное расстояние = 24.0 индекс центрального региона = 1320


In [8]:
# центральный регион

central = vor4.vertices[vor4.regions[v_min]]
#central

In [9]:
# нахожу длину ребра центрального политопа
d_edge_min = 100
count = 0
for i in range(0, len(central)-1):
    for j in range(i + 1, len(central)):
        d_edge = distance.euclidean(central[i], central[j])
        if d_edge_min >= d_edge:
            d_edge_min = d_edge
            count += 1
            
d_edge_min, count
                           

(1.0, 96)

In [ ]:
# индексы вершин центрального региона

vor4.regions[v_min]

In [ ]:
# список многоугольников

vor4.ridge_vertices

In [10]:

vor4.ridge_vertices[1]

[499, 500, 501, 508, 512, 514]

In [11]:
# трехмерные ячейки 
edge_central = []
for i in range(len(vor4.ridge_vertices)):
    count = 0

    for j in range(len(vor4.ridge_vertices[i])):
        if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
            count += 1 

    if count == len(vor4.ridge_vertices[i]):
        edge_central.append(vor4.ridge_vertices[i])
        
# перевожу индексы вершин в координаты
edge_central_coords = []
for i in range(len(edge_central)):
    r = []
    for j in range(len(edge_central[i])):
        r.append(vor4.vertices[edge_central[i][j]])
    edge_central_coords.append(np.array(r))
    
edge_central_coords[0]

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ]])

In [12]:
#количество трехмерных ячеек
len(edge_central_coords)

24

In [13]:
# составляем список граней многогранника

list_faces = []

# берем каждый 3х мерный многогранник
for m in edge_central_coords:
    
    # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
    list_edges = []
    for i in range(len(m)-1):
        for j in range(i+1, len(m)):
            k = []
            d = distance.euclidean(m[i], m[j])
            if d == 1:
                k.append(i)
                k.append(j)
                list_edges.append(k)
    # перебираем все комбинации ребер и ищем грани (если количество задействованных вершин в комбинации из 3х
    #  ребер = 3, то это грань)                       
    digits = list_edges
    list_faces_one = []  
    for var in itertools.combinations(digits, r=3):
        set_vet = set(var[0] + var[1] + var[2])
        set_vet_coord = []
        if len(set_vet) == 3:
            for i in range(len(set_vet)):
                set_vet_coord.append(m[list(set_vet)[i]])
            list_faces_one.append(set_vet_coord)
               
    list_faces.append(list_faces_one)
    
print('количество граней в одном зх мерном многограннике =', len(list_faces[1]))

количество граней в одном зх мерном многограннике = 8


In [14]:
central[0]

array([ 0.5, -0.5, -0.5, -0.5])

In [15]:
# диаметр многогранника (ищем максимальное расстояние между вершинами)

max_len = 0

for i in range(len(central) - 1):
    for j in range(i+1, len(central)):
        len_v = distance.euclidean(central[i], central[j])
        if len_v > max_len:
            max_len = len_v
            u1 = i
            u2 = j
               
print('диаметр =', max_len, 'индексы вершин:', u1, ',',  u2) 
# u1, u2 - индексы вершин, расстояние между которыми максимально

диаметр = 2.0 индексы вершин: 0 , 8


In [26]:
# координаты проекции

# coord1, coord2, coord3 - координаты 3 точек, d - точка, проекцию которой надо найти

def coords_proj(coord1, coord2, coord3, d):
    
    M = []        
    M.append(coord1)
    
    l1 = []
    l2 = []
    coords_proj
    for i in range(4):
        l1.append(coord2[i] - coord1[i])
        l2.append(coord3[i] - coord1[i])
        
    M.append(l1)
    M.append(l2)
    H1 = (np.array(M[1])).dot((np.array(M).T))
    K1 = np.array(l1).dot(d)
    H2 = (np.array(M[2])).dot((np.array(M).T))
    K2 = np.array(l2).dot(d)

    # Определение переменных
    x1, x2 = symbols('x1 x2')

    # Определение системы уравнений
    equations = [
        Eq(H1[0] + H1[1]*x1 + H1[2]*x2, K1),
        Eq(H2[0] + H2[1]*x1 + H2[2]*x2, K2)
    ]

    # Решение системы
    symbolic_solution = solve(equations, (x1, x2))

    # Вывод решения
    symbolic_solution
    
    coords = [1]
    coords.append(symbolic_solution[x1])
    coords.append(symbolic_solution[x2])
    
    
    return np.array(M).T.dot(coords)

In [27]:
# расстояние от точки до отрезка

import math

def distance_point_to_edge(p, a, b):
    
    # скалярное умножение 2х векторов
    def dot_product(v, w):
        sum_coord = 0
        for i in range(len(v)):
            sum_coord += v[i]*w[i]
        return sum_coord
    
    # разность 2х векторов
    def vector(v, w):
        list_dif = []
        for i in range(len(v)):
            k = w[i] - v[i]
            list_dif.append(k)
        return list_dif
    
    # длина вектора
    def magnitude(v):
        sum_sqr = 0
        for i in range(len(v)):
            sum_sqr += v[i]*v[i]
        return math.sqrt(sum_sqr)
    
    # умножение вектора на константу
    def scale(v, const):
        list_const = []
        for i in range(len(v)):
            list_const.append(v[i]*const)
        return list_const

    # сумма 2х векторов
    def add(v, w):
        sum_add = []
        for i in range(len(v)):
            sum_add.append(v[i]+w[i])
        return sum_add

    ab = vector(a, b)
    ap = vector(a, p)

    ab_magnitude = magnitude(ab)
    ap_dot_ab = dot_product(ap, ab)

    if ap_dot_ab <= 0:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке a
        return magnitude(ap)
    
    if ap_dot_ab >= ab_magnitude*ab_magnitude:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке b
        dist_b = 0
        for i in range(len(b)):
            dist_b += ((p[i] - b[i])**2)
        return math.sqrt(dist_b)

    # Проекция точки на отрезок находится внутри отрезка
    projection = add(a, scale(ab, ap_dot_ab/ab_magnitude**2))
    return magnitude(vector(p, projection))

In [28]:
# строим триангуляцию Делоне, используя ее, будем определять находится ли проекция внутри многогранника

delaunay = Delaunay(central)

In [29]:
max_len

2.0

In [30]:
min_d = 2 + np.sqrt(2)
min_d


3.414213562373095

In [31]:
min_d = 2.4

In [32]:
# ищет для подрешетки точку s
def s_point(sub_grid):

    coords4_1 = []
    digits = range(- 2, 2)
    coef = []
    
    # все возможные сочтания из чисел в диапазоне от -2 до 2
    for var in itertools.product(digits, repeat=4):
        coords4_1.append(np.dot(sub_grid.T, var).tolist())
        coef.append(var)
    
    min_dist = 100
    coords4_1.remove([0, 0, 0, 0]) #удаляю, чтобы не сравнивать с нулем
    
    # считаю все возмоожные расстояния между точками подрешетки и беру минимальное
    for i in range(len(coords4_1) - 1):
        for j in range(i + 1, len(coords4_1)):
            d_xy = distance.euclidean(coords4_1[i], coords4_1[j])
            
            if d_xy  >= min_d:
                continue
                #if d_xy < min_dist:
                    #min_dist = d_xy
                    #print(min_dist, coords4_1[i], coords4_1[j], c[i], c[j])
            else:
         
                return np.array([0, 0, 0, 0])
   # if ((min_dist - max_len) / max_len) > 1:

        
    # ищем координаты, которые есть в решетке и подрешетке   
    common_coord = []
    min_dist_i = 20
    for i in coords4:
        if i in coords4_1:
            common_coord.append(i)

    # расчитывам расстояние от [0.0, 0.0, 0.0, 0.0] до остальных точек и ищем минимальное расстояние
    min_dist = float('inf')
    for i in range(len(common_coord)):
        dist_centr = distance.euclidean(common_coord[i], [0.0, 0.0, 0.0, 0.0])

        if dist_centr < min_dist:

            min_dist = dist_centr 
            min_dist_i = i #расстояние от [0.0, 0.0, 0.0, 0.0] до ближайшего другого центра

    # находим точку s (середину между центрами многограников)
    if min_dist_i != 20:
        s = np.array(common_coord[min_dist_i], float) * 0.5
        #print(common_coord[min_dist_i])
        #for k in range(len(coords4_1)):
           # if coords4_1[k] == common_coord[min_dist_i]:
                #print(coef[k])
        

        return s
        #else:
    return np.array([0, 0, 0, 0])




In [33]:
# ищет для подрешетки запрещенное расстояние

def dist(s):
    min_dist_s = float('inf') #минимальное расстояние до точки s 
    
    
    # перебирем все грани
    for i in range(len(list_faces)):

        for j in range(len(list_faces[i])):

            # находим координаты проекции на грань и расстояние от точки до проекции
            f_coords = coords_proj(list_faces[i][j][0], list_faces[i][j][1], list_faces[i][j][2], s)
            f = distance.euclidean(s, np.array(f_coords, dtype = 'float'))
            

            # если расстояние меньше, то проверяем находится ли точка проекции в триангуляции
            # если да, то приравниваем минимальное расстояние к расстоянию между точкой и ее проекцией
            if f < min_dist_s:            
                simplex = delaunay.find_simplex(f_coords)
                
                if simplex != -1:                    
                    min_dist_s = f
  
              
                
    # ищем расстояние до каждого ребра через функцию distance_point_to_edge 
    #for i in range(len(edge_central_coords)):
       # digits = edge_central_coords[i]
        #for var in itertools.combinations(digits, r=2):
           # dist_point = distance_point_to_edge(s, var[0], var[1])

            # если полученное расстояние меньше минимального, то меняем значение минимального расстояния
           # if dist_point < min_dist_s:
             #   min_dist_s = dist_point

                
    return (min_dist_s * 2)/ max_len


In [34]:
dist(np.array((0, 1, 0, 1)))

0.816496580927726

In [35]:
len(list_faces)

24

In [36]:
list_faces[0]

[[array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])]]

In [37]:
def gram_schmidt(v):

    w_1 = copy.deepcopy(v[0])
    w_n = copy.deepcopy(w_1)
    w_array = [deepcopy(w_n)]
    
    for n in range(1, len(v)):
        v_n = copy.deepcopy(v[n])
        w_n = copy.deepcopy(v_n)

        for j in range(n):
            w_j = deepcopy(w_array[j])
            if not any(w_j):
                continue
            w_n -= np.dot(v_n, w_j) / np.dot(w_j, w_j) * w_j
        
        w_array += [w_n]
          
    return w_array

In [38]:
d = 0.75

def mu(b_i, b_j):
    return np.dot(b_i, b_j) / np.dot(b_j, b_j) if np.dot(b_j, b_j) != 0 else 0


def LLL(l_basis):

    ortho = gram_schmidt(l_basis)

    k = 1
    n = len(ortho)

    while k < n:
        for j in range(k - 1, -1, -1):
            proj = mu(l_basis[k], ortho[j])
            if abs(proj) > 1/2:
                l_basis[k] -= l_basis[j] * round(proj)
                ortho = copy.deepcopy(gram_schmidt(l_basis))
            #print(l_basis, 111)

        if np.dot(ortho[k], ortho[k]) >= (d - (mu(l_basis[k], ortho[k-1]))**2) * (np.dot(ortho[k-1], ortho[k-1])):
            #print(l_basis, 222)
            k += 1

        else:
            s = copy.deepcopy(l_basis[k-1])
            l_basis[k-1] = copy.deepcopy(l_basis[k])
            l_basis[k] = copy.deepcopy(s)
            ortho = gram_schmidt(l_basis)
            k = max(k-1, 1)
            #print(l_basis, 333)

    return l_basis

In [39]:
#list_mat_s = dict() # ключ - точка s, значение - списоок матриц

dict_mat_s1_alg = {} # ключ - mat, значение - точка s1 по алгоритму s1
dict_lll_s1 = {} # ключ - приведенный базис, значение - точка s1 как половина от кратчайшего вектора lll-базиса
dict_lll_s1_alg = {} # ключ - приведенный базис, значение - точка s1 по алгоритму s1
dict_mat_lll = {} #  ключ - mat, значение - lll
#dict_dist = defaultdict(list)

list_mat = []
list_lll = []
list_s1 = []
list_s1_alg = []

# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 6]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
   
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                        
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            print(mat, s1)
  
                            list_mat.append(mat)
                            list_lll.append(basis)
                            s1_2 = s_point(sub_grid)
                            print(s1_2)
                            list_s1.append(s1)
                            list_s1_alg.append(s1_2)


0
[[1. 1. 1. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 6.]] [-0.5  1.  -0.5  0. ]
[-1.   0.5  0.5  0. ]
[[1. 2. 1. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 6.]] [-1.   0.5 -0.5  0. ]
[-0.5  1.   0.5  0. ]
[[1. 1. 2. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 6.]] [0.5 1.  0.5 0. ]
[-0.5  0.5  1.   0. ]
[[1. 0. 0. 1.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 6.]] [1.5 0.  0.  0.5]


KeyboardInterrupt: 

In [69]:
#list_mat_s = dict() # ключ - точка s, значение - списоок матриц

dict_mat_s1_alg = {} # ключ - mat, значение - точка s1 по алгоритму s1
dict_lll_s1 = {} # ключ - приведенный базис, значение - точка s1 как половина от кратчайшего вектора lll-базиса
dict_lll_s1_alg = {} # ключ - приведенный базис, значение - точка s1 по алгоритму s1
dict_mat_lll = {} #  ключ - mat, значение - lll
#dict_dist = defaultdict(list)

list_mat = []
list_lll = []
list_s1 = []
list_s1_alg = []

# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 6]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
                        print(basis)
                        
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                        
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            print(mat, s1)
                            #dict_lll_s1[basis] = s1
                            list_mat.append(mat)
                            list_lll.append(basis)
                            s1_2 = s_point(sub_grid)
                            print(s1_2)
                            list_s1.append(s1)
                            list_s1_alg.append(s1_2)
                            #dict_lll_s1_alg[basis] = s1_2
                            #dict_mat_s1_alg[mat] = s1_2

0
[[ 2.  0.  0.  0.]
 [-1.  3.  0.  0.]
 [-1.  0.  3.  0.]
 [ 0.  0.  0.  6.]]
[[ 0.  2.  0.  0.]
 [ 3.  1.  0.  0.]
 [ 0. -1.  3.  0.]
 [ 0.  0.  0.  6.]]
[[-1.  1.  0.  0.]
 [ 3.  3.  0.  0.]
 [ 1.  2.  3.  0.]
 [ 0.  0.  0.  6.]]
[[ 0.  0.  2.  0.]
 [ 3.  0.  1.  0.]
 [ 0.  3. -1.  0.]
 [ 0.  0.  0.  6.]]
[[-1.  2. -1.  0.]
 [-1. -1.  2.  0.]
 [ 2.  2.  2.  0.]
 [ 0.  0.  0.  6.]]
[[1. 1. 1. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 6.]] [-0.5  1.  -0.5  0. ]


KeyboardInterrupt: 

list_mat_s = dict() # ключ - точка s, значение - списоок матриц
list_val = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
#list_val = [0, 1, 2, 3, 4, 5]
#det_list = [] # ключ - точка s1,  значение список подрешеток, соответствующих ютой точке 
lll_good = [] # хорошие приведенные базисы
lll_bad = [] # плохие приведенные базисы
dict_lll = {} # ключ - приведенный базис, значение - точка s1
dict_dist = defaultdict(list)

# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 6]], float)


for i1 in list_val:
    mat[2][3] = i1
    print(i1)
    for i2 in list_val:
        mat[1][3] = i2
        for i3 in list_val:
            mat[1][2] = i3
            for i4 in list_val:
                mat[0][3] = i4
                for i5 in list_val:
                    mat[0][2] = i5
                    for i6 in list_val:
                        mat[0][1] = i6
                        #print(mat)
                        sub_grid = np.dot(mat, grid)
                        #print(sub_grid, 66)
                        #basis = gram_schmidt(sub_grid)
                        basis = LLL(sub_grid)
                        #print(basis)
                        #if any(all(np.array_equal(datum, q) for datum, q in zip(data, basis)) for data in lll_good):   
                            #print(1)
                           # list_mat_s[tuple(list_ll_s1[basis.tobytes()])].append([copy.deepcopy(mat)])

                       # else:
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i

                        #print(min_dist)
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            #print(mat, s1)
                            #s1 = s_point(basis)

                            dist_s1 = dist(s1)
                            #print(s1, mat, dist_s1)
                            if dist_s1 > 1:
                                print(s1, basis, min_dist, mat)
                                #det_list.append(det_mat)

                            lll_good.append(basis)
                            dict_lll[basis.tobytes()] = copy.deepcopy(s1)
                            dict_dist[dist_s1] = mat
                            if tuple(s1) in list_mat_s:
                                list_mat_s[tuple(s1)].append([copy.deepcopy(mat)])

                            else:
                                list_mat_s[tuple(s1)] = [copy.deepcopy(mat)]


list_mat_s = dict() # ключ - точка s, значение - списоок матриц
list_val = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
#list_val = [0, 1, 2, 3, 4, 5]
#det_list = [] # ключ - точка s1,  значение список подрешеток, соответствующих ютой точке 
lll_good = [] # хорошие приведенные базисы
lll_bad = [] # плохие приведенные базисы
dict_lll = {} # ключ - приведенный базис, значение - точка s1
dict_dist = defaultdict(list)

# задаю величину определителя
det_mat = 32
mat = np.array([[2, 0, 0, 0], [0, 2, 0, 0], [0, 0, 2, 0], [0, 0, 0, 4]], float)


for i1 in list_val:
    mat[2][3] = i1
    print(i1)
    for i2 in list_val:
        mat[1][3] = i2
        for i3 in list_val:
            mat[1][2] = i3
            for i4 in list_val:
                mat[0][3] = i4
                for i5 in list_val:
                    mat[0][2] = i5
                    for i6 in list_val:
                        mat[0][1] = i6
                        #print(mat)
                        sub_grid = np.dot(mat, grid)
                        #print(sub_grid, 66)
                        #basis = gram_schmidt(sub_grid)
                        basis = LLL(sub_grid)
                        #print(basis)
                        #if any(all(np.array_equal(datum, q) for datum, q in zip(data, basis)) for data in lll_good):   
                            #print(1)
                           # list_mat_s[tuple(list_ll_s1[basis.tobytes()])].append([copy.deepcopy(mat)])

                       # else:
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i

                        #print(min_dist)
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            print(mat, s1)
                            #s1 = s_point(basis)

                            dist_s1 = dist(s1)
                            #print(s1, mat, dist_s1)
                            if dist_s1 > 1:
                                print(s1, basis, min_dist, mat)
                                #det_list.append(det_mat)

                            lll_good.append(basis)
                            dict_lll[basis.tobytes()] = copy.deepcopy(s1)
                            dict_dist[dist_s1] = mat
                            if tuple(s1) in list_mat_s:
                                list_mat_s[tuple(s1)].append([copy.deepcopy(mat)])

                            else:
                                list_mat_s[tuple(s1)] = [copy.deepcopy(mat)]


In [ ]:
# находим минимальный определитель для каждой точки s и матрицы, соответствующие данному хроматическому 
# числу для каждой точки s

dict_det = {} # ключ - точка s, значение - минимальное хроматическое число среди всех матриц
dict_s = {} # ключ - точка s, значение - матрицы, соответствующие минимальному хроматическому числу


# перебираем все точки s и для каждой считаем определитель, находим минимальный и сооветствующие матрицы
for j in list_mat_s:
    best_mat = []
    min_det = float('inf')
    for i in list_mat_s[j]:

        det = round(abs(float(np.linalg.det(i))), 0)


        # если находим меньший подходящий определитель, то очищаем список матриц и записываем новое з
        # начение минимального определителя
        if det == min_det:
            best_mat.append(i)
        if det > 0 and det < min_det:

            best_mat.clear()
            best_mat.append(i)
            min_det = det
  
    # если определитель меньше бесконечности, то добавляем полученные значения в словари
    if min_det < 81:
        dict_s[j] = best_mat
        dict_det[j] = min_det

print(len(dict_s), dict_s)

In [ ]:
# ищем расстояние для каждой точки s

dict_dist = {}
for key in dict_s:
    d = dist(key)
        
    dict_dist[key] = d

In [ ]:
dict_dist

In [ ]:
# меняем тип данных в словаре с хроматичекими числами

dict_det = {k: int(v) for k, v in dict_det.items()}
dict_det

In [ ]:
# создаю датафрейм с колонками: 's' - координаты точки s, 'ch_n' - минимальное хроматическое число, 
# 'dist' - максимальное запрещенное расстояние, 'mats' - список соответствующих матриц

df = pd.DataFrame(list(dict_det.items()), columns=['s', 'ch_n'])
df['dist'] = df['s'].map(dict_dist)
df['mats'] = df['s'].map(dict_s)
df

In [ ]:
df.loc[(df['dist'] > 1)]
